In [1]:
import pandas as pd
import openai
import csv
import time
import os
import regex as re
import math
import random

import openai

pd.set_option('display.max_colwidth', None)
file_path = "D:/api_key.txt"
with open(file_path, 'r') as file:
    api_key = file.read()

openai.api_key = (api_key)

In [2]:
def genereer_data(voorbeeld_zin, voorbeeld_zin2,temp=1,top_p=0.9):
    generate_faulty = "Genereer medische tekst met grammaticale en spellingsfouten. De tekst beschrijft dagelijkse gebeurtenissen van de client onder zorg. Maak zeer diverse tekst. Bijvoorbeeld: " + voorbeeld_zin + "Bijvoorbeeld: " + voorbeeld_zin2
    message_history = []
    message_history.append({'role' : "system", "content": generate_faulty})
    message_history.append({'role' : "system", "content" : 'genereer 1 tekst'})
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',messages=message_history,
        temperature=temp,
        top_p=top_p
    )
    #write header to file. 
    #write original sentence to file and correction.
    reply_content = completion.choices[0].message.content
    return reply_content

In [3]:
def genereer_data(voorbeeld_zin, voorbeeld_zin2, temp =1, top_p=0.9, n=5, pp=0):
    generate_correct = "Genereer medische tekst. De tekst is een kort en bondig geschreven en brengt de essentie over van relevante gebeurtenissen van de dag of dagdeel. Het gaat over een oudere client die zorg ontvangt. De tekst is geschreven door een zorgprofessional die geen moeilijke woorden gebruikt. Bijvoorbeeld: " + voorbeeld_zin + "Bijvoorbeeld: " + voorbeeld_zin2
    message_history = []
    generate_1 = 'Genereer 1 tekst'
    message_history.append({'role' : "system", "content": generate_correct})
    message_history.append({"role": "user", 'content' : generate_1})

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',messages=message_history,
        temperature=temp,
        top_p=top_p,
        presence_penalty=0,
        n=n
    )
    return completion

In [4]:
def sample_from_df(df):
    random_row = df.sample()
    # extract the values of the sampled row
    values = random_row.values[0]
    # print the values as a string
    voorbeeld_zin = (''.join(map(str, values)))

    same = True
    while same == True:
        random_row = df.sample()
        # extract the values of the sampled row
        values = random_row.values[0]
        # print the values as a string
        voorbeeld_zin2= (''.join(map(str, values)))
        if voorbeeld_zin != voorbeeld_zin2:
            same = False

    

    return voorbeeld_zin, voorbeeld_zin2

In [5]:
def generate_correct_data(df, output_file_path, amount_per_combination, temp, top_p, generation_limit, pp):
    # check if output file already exists
    if os.path.exists(output_file_path):
        # if it does, read the last row that was successfully processed
        with open(output_file_path, 'r', encoding='utf-8') as output_file:
            reader = csv.reader(output_file)
            for row in reader:
                pass
            last_row_processed = reader.line_num
    else:
        # if it doesn't, set last_row_processed to 0
        last_row_processed = 0

    # open input and output files
    #ISO-8859-1
    with open(output_file_path, 'a', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)

        #If initializing a new document, skip the header and rewrite the new format.
        if last_row_processed == 0:
            writer.writerow(['Generated', 'Example_1', 'Example_2', 'Temperature', "Top_p", "Pres_penalty"])
        number_generated = 0
        current_time = 100
        start_time = 10
        while generation_limit > number_generated:
            time.sleep(5)
            current_time = time.time()
            if current_time - start_time >= 62:

                voorbeeld_zin, voorbeeld_zin2 = sample_from_df(df)
                #print(reader.line_num)
                try:
                    start_time = time.time()
                    completion = genereer_data(voorbeeld_zin, voorbeeld_zin2, n = amount_per_combination, temp=temp, top_p=top_p )
                    #print(reply_content)
                except:
                    # if API call fails, wait for 5 seconds and retry
                    print('failed... retrying in 5 seconds')
                    time.sleep(5)
                    #Longer prompt costs more.
                    start_time = time.time()
                    completion = genereer_data(voorbeeld_zin, voorbeeld_zin2, n = amount_per_combination, temp=temp, top_p=top_p )
                    
                for i in range(len(completion.choices)):
                    writer.writerow([completion.choices[i].message.content, voorbeeld_zin, voorbeeld_zin2, temp, top_p, pp])
                    number_generated += 1
                    print('Rows Processed:', number_generated)

                # flush output file buffer to ensure data is saved
                output_file.flush()

                # print progress
                #print(f"Processed row {last_row_processed}")
                # break out of loop if num_rows is reached
                if generation_limit is not None and number_generated >= generation_limit:
                    print('Generation limit reached')
                    break


In [6]:
df = pd.read_csv("hand_made_correct_medical.csv", sep=',', encoding='ISO-8859-1', header=None)
generate_correct_data(df=df, output_file_path='E:\Data_exploration/GitHub/paraphraser_code/data/generated/generate_correct_medical_v1.csv', amount_per_combination=5, temp=1, top_p=0.9, generation_limit=1, pp=0)

Rows Processed: 1
Rows Processed: 2
Rows Processed: 3
Rows Processed: 4
Rows Processed: 5
Generation limit reached


In [7]:
df2 = pd.read_csv('E:\Data_exploration/GitHub/paraphraser_code/data/generated/generate_correct_medical_v1.csv', header=0, encoding='utf-8')

In [8]:
df2.head(20)#['Generated'][1]

,Generated,Example_1,Example_2,Temperature,Top_p,Pres_penalty
0,"Meneer Jansen was vandaag wat vermoeid en had wat moeite met opstaan. Ik heb hem geholpen met zijn ochtendroutine en hem even gezelschap gehouden tijdens het wassen en aankleden. Hij had geen zin om te douchen, maar wel om zich op te frissen met een washandje. Daarna heb ik hem geholpen met het aantrekken van schone kleren. Meneer Jansen wilde graag op zijn kamer ontbijten, dus heb ik zijn ontbijt daar naartoe gebracht. We hebben gezellig gekletst terwijl hij rustig at. Na het ontbijt heeft meneer Jansen even gerust op zijn bed en daarna hebben we samen een kleine wandeling gemaakt door de gangen. Hij genoot van het bewegen en het praatje onderweg. Het was fijn om te zien dat hij zich wat beter voelde na de rust en de wandeling.",Gekletst met meneer tijdens de algemene dagelijkse levensverrichtingen. Hij heeft op zijn kamer gegeten. Meneer schone kleren aangedaan.,Mevrouw stond om 9.00 op de gang. Ze wilde zich wel aankleden maar niet douchen. Ze zei dat ik hier niet over moest zeuren. Mevrouw aangekleed en naar het ontbijtbuffet gebracht.,1,0.9,0
1,"Meneer Jansen had vanochtend een goede dag. Hij was al vroeg wakker en stond te wachten op de gang. Samen hebben we gepraat terwijl ik hem hielp met wassen en aankleden. Hij wilde zich niet douchen, maar dat vond hij geen probleem. Daarna heeft meneer Jansen zelf zijn schone kleren aangetrokken. Ik heb hem naar het ontbijtbuffet gebracht waar hij lekker heeft gegeten.",Gekletst met meneer tijdens de algemene dagelijkse levensverrichtingen. Hij heeft op zijn kamer gegeten. Meneer schone kleren aangedaan.,Mevrouw stond om 9.00 op de gang. Ze wilde zich wel aankleden maar niet douchen. Ze zei dat ik hier niet over moest zeuren. Mevrouw aangekleed en naar het ontbijtbuffet gebracht.,1,0.9,0
2,Mevrouw Jansen stond om 9.00 uur op de gang. Ze wilde zich wel aankleden maar niet douchen. Ze zei dat ik hier niet over moest zeuren. Ik heb mevrouw geholpen met aankleden en haar naar het ontbijtbuffet gebracht.,Gekletst met meneer tijdens de algemene dagelijkse levensverrichtingen. Hij heeft op zijn kamer gegeten. Meneer schone kleren aangedaan.,Mevrouw stond om 9.00 op de gang. Ze wilde zich wel aankleden maar niet douchen. Ze zei dat ik hier niet over moest zeuren. Mevrouw aangekleed en naar het ontbijtbuffet gebracht.,1,0.9,0
3,"Meneer Jansen had vandaag een rustige dag. Ik heb hem geholpen met de dagelijkse verzorging en we hebben even gekletst tijdens het eten. Hij wilde graag op zijn kamer eten, dus ik heb zijn maaltijd daar naartoe gebracht. Daarna heeft hij schone kleren aangedaan. Het ging allemaal prima en hij was tevreden.",Gekletst met meneer tijdens de algemene dagelijkse levensverrichtingen. Hij heeft op zijn kamer gegeten. Meneer schone kleren aangedaan.,Mevrouw stond om 9.00 op de gang. Ze wilde zich wel aankleden maar niet douchen. Ze zei dat ik hier niet over moest zeuren. Mevrouw aangekleed en naar het ontbijtbuffet gebracht.,1,0.9,0
4,"Vanochtend heb ik met meneer gesproken tijdens de dagelijkse bezigheden. Hij heeft zijn maaltijd op zijn kamer gegeten. Daarna heb ik hem schone kleren aangetrokken.\n\nMevrouw stond om 9.00 uur op de gang. Ze wilde zich wel aankleden, maar niet douchen. Ze vertelde me dat ik er niet over moest zeuren. Ik heb haar geholpen met aankleden en naar het ontbijtbuffet gebracht.",Gekletst met meneer tijdens de algemene dagelijkse levensverrichtingen. Hij heeft op zijn kamer gegeten. Meneer schone kleren aangedaan.,Mevrouw stond om 9.00 op de gang. Ze wilde zich wel aankleden maar niet douchen. Ze zei dat ik hier niet over moest zeuren. Mevrouw aangekleed en naar het ontbijtbuffet gebracht.,1,0.9,0
